In [5]:
from datetime import date, datetime
from dateutil.relativedelta import relativedelta

# Lê a data de corte
data_base_str = spark.sql("""
    SELECT vDataCorte FROM lk_ControleProcessos.tab_controle_incremental
""").collect()[0]['vDataCorte']

# Converte para date
data_base = datetime.strptime(data_base_str, "%Y-%m-%d").date()

# Força o dia para 1º do mês
data_inicio_mes = data_base.replace(day=1)

# Formata como string para o SQL
data_limite_str = data_inicio_mes.strftime("%Y-%m-%d")

# Configuração para evitar erro de datas antigas
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInRead", "CORRECTED")
spark.conf.set("spark.sql.legacy.parquet.datetimeRebaseModeInWrite", "CORRECTED")

# Atualiza datas inválidas
spark.sql("""
    UPDATE tar_proces
    SET start_date = DATE('2000-01-01')
    WHERE start_date IS NOT NULL
      AND start_date < DATE('1900-01-01')
""")

# Verifica datas após a atualização
spark.sql("""
    SELECT MIN(start_date) AS DATA_MINIMA, MAX(start_date) AS DATA_MAXIMA
    FROM tar_proces
""").show()

# Executa o DELETE
spark.sql(f"""
    DELETE FROM tar_proces
    WHERE start_date >= DATE('{data_limite_str}')
""")

StatementMeta(, 710b1cc0-2716-4058-b6a2-3bf1fa90398f, 7, Finished, Available, Finished)

+-------------------+-------------------+
|        DATA_MINIMA|        DATA_MAXIMA|
+-------------------+-------------------+
|2015-03-04 14:03:55|2025-07-24 03:41:14|
+-------------------+-------------------+



DataFrame[num_affected_rows: bigint]